In [6]:
import os
import sys

In [7]:
project_root=os.path.abspath(os.path.join(os.getcwd(),".."))

In [8]:
print(project_root)

d:\d_data\AnuragReddy_Naredla_python_data\python\data_science_concepts_github\LLMOPS\project3\automated-research-and-report-generation


In [9]:
sys.path.append(project_root)

In [10]:

from research_and_analysts.utils.model_loader import ModelLoader